# Capture Landmarks and Export to CSV

In [1]:
import csv
landmarks = ["class"]
for val in range(0, 20 + 1): # hand have 21 landmarks
    landmarks += [f"x{val}", f"y{val}", f"z{val}"]
with open("coords.csv", mode = "w", newline = "") as f:
    csv_writer = csv.writer(f, delimiter = ",", quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time
import csv

class_name = "3" 
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
window_name = "Mediapipe"
cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)
# cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
WIDTH, HEIGHT = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

font = cv2.FONT_HERSHEY_SIMPLEX

count_flag = 0 # time countdown when flag = 0, time count when flag = 1,

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    time_start = time.time() # calculate time
    while True:
        ret, image = cap.read()
        
        image = cv2.flip(image, 1) 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # image.flags.writeable = False

        results = hands.process(image) # Make detection

        # image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
                hand_row = list (np.array([[i.x, i.y, i.z] for i in hand_landmarks.landmark]).flatten())
            hand_row.insert(0, class_name)
            with open("coords.csv", mode = "a", newline = "") as f:
                csv_writer = csv.writer(f, delimiter = ",", quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                csv_writer.writerow(hand_row)
        
            # timing and show it
            time_count = time.time() - time_start

            if count_flag == 0:
                if time_count <= 3:
                    time_count = 3 - time_count
                else :
                    count_flag = 1
                    time_start = time.time()    
                    time_count = 0
            
        text = str(int(time_count))
        # get boundary of this text
        textsize = cv2.getTextSize(text, font, 4, 2)[0]
        
        # get coords based on boundary
        textX = int((WIDTH - textsize[0]) / 2)
        textY = int((HEIGHT + textsize[1]) / 2)
        cv2.putText(image, text, (textX, textY),
                    font, 4, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.putText(image, "class " + class_name, (0, textsize[1]), font, 2, (0,0,0), 2, cv2.LINE_AA) # show the class at left top
        
        cv2.imshow(window_name, image)
        
        if ret == False or cv2.waitKey(10) == ord("q") or time_count > 15: # get frature in 15's
            break

    cap.release()
    cv2.destroyAllWindows()